In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

import warnings
warnings.filterwarnings('ignore')

In [4]:

from IPython.display import clear_output
from datetime import datetime
import humanize
import os
import re
import time
import winsound

duration = 1000  # milliseconds
freq = 880  # Hz
print(f'Last run on {datetime.now()}')

Last run on 2023-02-26 19:53:20.076462



----

In [6]:

# Show what's in the database already for this html string
def do_cypher_tx(tx):
    cypher_str = '''
        MATCH (np:NavigableParents)
        WHERE
            (np.is_header = 'False')
            AND (np.is_task_scope = 'True')
        ''' + cu.return_everything_str + ';'
    results_list = tx.run(query=cypher_str)

    return [dict(record.items()) for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx)
df = pd.DataFrame(row_objs_list[:3])

[{'navigable_parent': '•\xa0\xa0 \xa0Work closely with our media agency to marry the front end media performance to back end, Client owned measurements including site analytics, conversions and sentiment.', 'is_header': 'False', 'is_task_scope': 'True', 'is_qualification': None, 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'False', 'is_legal_notification': 'False', 'is_job_title': 'False', 'is_office_location': 'False', 'is_job_duration': 'False', 'is_supplemental_pay': 'False', 'is_educational_requirement': 'False', 'is_interview_procedure': 'False', 'is_corporate_scope': 'False', 'is_posting_date': 'False', 'is_other': 'False'}, {'navigable_parent': '•\xa0\xa0 \xa0Maintain and further optimize a set of dashboards that provide data-informed insights as it pertains to keywords, audiences, lengths and investment levels.', 'is_header': 'False', 'is_task_scope': 'True', 'is_qualification': None, 'is_minimum_qualification': 'False', 'is_preferred_qualification': 'Fals


----

In [12]:

from nltk.tokenize import sent_tokenize

roles_df = s.load_csv(csv_name='Accenture_Technology_Open_Roles_02.21.2023', folder_path='../saves').rename(columns={'Client': 'Client Name'})
roles_df.columns = [re.sub(r'[^A-Za-z0-9]+', ' ', cn).strip().replace(' ', '_').lower() for cn in roles_df.columns]
mask_series = (roles_df.role_is_sold == 'Yes')
base_dict = {
    'header': 'False',
    'task_scope': 'False',
    'minimum_qualification': 'False',
    'preferred_qualification': 'False',
    'educational_requirement': 'False',
    'legal_notification': 'False',
    'other': 'False',
    'corporate_scope': 'False',
    'job_title': 'False',
    'office_location': 'False',
    'job_duration': 'False',
    'supplemental_pay': 'False',
    'interview_procedure': 'False',
    'posting_date': 'False'
}
fake_stops_list = ['e.g.', 'etc.', 'M.S.', 'B.S.', 'Ph.D.', '(ex.', '(Ex.', 'U.S.',
                   'i.e.', '&amp;']
replacements_list = ['eg', 'etc', 'MS', 'BS', 'PhD', '(eg', '(eg', 'US', 'ie', '&']

In [14]:

def create_accenture_file(file_path, row, verbose=False):
    with open(file_path, 'w', encoding='utf-8') as f:
        print('<html><body><div id="jobDescriptionText" class="jobsearch-jobDescriptionText">', file=f)

        # Get dual-roled descriptions list
        role_description = row.role_description.strip()
        paragraph_str = re.sub(r'\s+', ' ', role_description).strip()
        for fake_stop, replacement in zip(fake_stops_list, replacements_list):
            paragraph_str = paragraph_str.replace(fake_stop, replacement)
        child_strs_list = sent_tokenize(paragraph_str)
        
        # Get parts-of-speech list
        child_tags_list = ha.get_child_tags_list(child_strs_list)
        is_header_list = []
        for is_header, child_str in zip(ha.get_is_header_list(child_strs_list), child_strs_list):
            if is_header is None:
                probs_list = lru.ISHEADER_PREDICT_PERCENT_FIT(child_str)
                idx = probs_list.index(max(probs_list))
                is_header = [True, False][idx]
            is_header_list.append(is_header)
        feature_dict_list = hc.get_feature_dict_list(child_tags_list, is_header_list, child_strs_list)
        feature_tuple_list = []
        for feature_dict in feature_dict_list:
            feature_tuple_list.append(hc.get_feature_tuple(feature_dict, lru.pos_lr_predict_single))
        crf_list = crf.CRF.predict_single(crf.sent2features(feature_tuple_list))
        db_pos_list = []
        for navigable_parent in child_strs_list:
            db_pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list=db_pos_list)
        for crf_symbol, db_symbol in zip(crf_list, db_pos_list):
            if db_symbol in [None, 'O', 'H']:
                pos_list.append(crf_symbol)
            else:
                pos_list.append(db_symbol)
        if verbose:
            print(pos_list)
            raise

        # Role Primary Skill
        role_primary_skill = row.role_primary_skill.strip()
        print('<hrq>Role Primary Skill:</hrq>', file=f)
        for role_str in child_strs_list:
            if (' plus' in role_str) or ('preferred' in role_str):
                print(f'<opq>{role_str}</opq>', file=f)
            else:
                role_str = role_str.lstrip('·*- ')
                if role_str:
                    print(f'<orq>Ability to {role_str[0].lower()}{role_str[1:]}</orq>', file=f)
        skills_list = [s.strip() for s in re.split('[|/]', role_primary_skill, 0) if s.strip()]
        for skill_str in skills_list:
            print(f'<orq>{skill_str}</orq>', file=f)

        # Role Description
        print('<hts>Role Description:</hts>', file=f)
        for role_str in child_strs_list:
            print(f'<ots>{role_str}</ots>', file=f)

        # Role ID
        print(f'<ojt>Role ID: {role_id}</ojt>', file=f)

        # Client
        print(f'<ocs>Client: {client_name}</ocs>', file=f)

        # Role Title
        print(f'<ojt>Role Title: {role_title}</ojt>', file=f)

        # Assigned Role
        assigned_role = row.assigned_role.strip()
        print(f'<ots>Assigned Role: {assigned_role}</ots>', file=f)

        # Project Metro City
        project_metro_city = row.project_metro_city.strip()
        print(f'<ool>Project Metro City: {project_metro_city}</ool>', file=f)

        # Career Level From - To
        career_level_from_to = row.career_level_from_to.strip()
        print(f'<osp>Career Level From - To: {career_level_from_to}</osp>', file=f)

        # Role Start Date
        role_start_date = row.role_start_date.strip()
        print(f'<ojd>Role Start Date: {role_start_date}</ojd>', file=f)

        # Role End Date
        role_end_date = row.role_end_date.strip()
        print(f'<ojd>Role End Date: {role_end_date}</ojd>', file=f)

        # Role Client Supply Contact
        role_client_supply_contact = row.role_client_supply_contact.strip()
        print(f'<oip>Role Client Supply Contact: {role_client_supply_contact}</oip>', file=f)

        # Role Primary Contact
        role_primary_contact = row.role_primary_contact.strip()
        print(f'<oip>Role Primary Contact: {role_primary_contact}</oip>', file=f)

        # Role Primary Contact (Email ID)
        role_primary_contact_email_id = row.role_primary_contact_email_id.strip()
        print(f'<oip>Role Primary Contact (Email ID): {role_primary_contact_email_id}</oip>', file=f)

        # Role Is Sold
        role_is_sold = row.role_is_sold.strip()
        print(f'<oip>Role Is Sold: {role_is_sold}</oip>', file=f)

        print('</div></body></html>', file=f)

In [ ]:

STARTSWITH_STR = '''
    MATCH (np:NavigableParents)
    WHERE (np.navigable_parent STARTS WITH "<{html_prefix}{html_suffix}>")
    SET
        '''
COMMA_STR = ''',
        '''

In [ ]:

for row in tqdm(roles_df[mask_series].itertuples(index=False), total=len(roles_df[mask_series])):
    
    # Create the file path
    role_id = str(row.role_id).strip()
    client_name = row.client_name.strip()
    role_title = row.role_title.strip()
    file_name = re.sub(r'[^A-Za-z0-9]+', ' ', f'{role_id} {client_name} {role_title}').strip().replace(' ', '_') + '.html'
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    # if os.path.exists(file_path):
    #     cu.delete_filename_node(file_name, verbose=False)
    if not os.path.exists(file_path):
        
        # Create the file using HTML
        create_accenture_file(file_path, row, verbose=True)
        
        # Populate the database with the file info
        page_soup = wsu.get_page_soup(file_path)
        div_soup = page_soup.find_all(name='div', id='jobDescriptionText')[0]
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.ensure_filename(file_name, verbose=False)
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
        cu.set_accenture_data(
            file_name, assigned_role, career_level_from_to, client_name, project_metro_city, role_client_supply_contact, role_end_date,
            role_id, role_is_sold, role_primary_contact, role_primary_contact_email_id, role_start_date, role_title, verbose=False
        )
        for is_header, html_prefix in zip(['True', 'False'], ['h', 'o']):
            for cypher_suffix, html_suffix in zip(
                ['corporate_scope', 'interview_procedure', 'job_duration', 'job_title', 'office_location', 'posting_date', 'preferred_qualification', 'minimum_qualification', 'supplemental_pay', 'task_scope'],
                ['cs', 'ip', 'jd', 'jt', 'ol', 'pd', 'pq', 'rq', 'sp', 'ts']
            ):
                working_dict = base_dict.copy()
                working_dict.update({'header': is_header})
                working_dict.update({cypher_suffix: 'True'})
                
                def do_cypher_tx(tx, verbose=False):
                    cypher_str = STARTSWITH_STR.format(html_prefix, html_suffix)
                    attrs_list = [f"np.is_{k} = '{v}'" for k, v in working_dict.items()]
                    cypher_str += COMMA_STR.join(attrs_list)
                    if verbose:
                        clear_output(wait=True)
                        print(cypher_str)
                    parameter_dict = {}
                    results_list = tx.run(query=cypher_str, parameters=parameter_dict)

                with cu.driver.session() as session:
                    session.write_transaction(do_cypher_tx, verbose=False)
        clear_output(wait=True)
        print(file_name)


----

In [ ]:

roles_df['Project Metro City'].map(lambda x: 'remote' in x.lower()).any()

In [ ]:

mask_series = roles_df['Role Description'].map(lambda x: 'remote' in x.lower())
roles_df[mask_series]['Role Description'].tolist()

In [ ]:

print()
for column_name in roles_df.columns:
    variable_name = column_name.lower().replace(' ', '_')
    print(f"""        {variable_name} = row_series['{column_name}'].strip()""")


----

In [ ]:

files_list = sorted([fn for fn in os.listdir(cu.SAVES_HTML_FOLDER) if fn.endswith('.html')])

In [ ]:

t0 = time.time()
for file_name in files_list:
    cu.ensure_filename(file_name, verbose=True)
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t0, minimum_unit='seconds', format='%0.0f')
winsound.Beep(freq, duration)
print(f'{len(files_list):,} file names reinserted in {duration_str}')


----
## Unless you have written consent from the Generative AI and LLM CoE, you may not use generative AI tools while coding and cannot upload Accenture, ecosystem or client content or data to these tools.

In [ ]:

import pandas as pd
from tqdm import tqdm
from selenium.common.exceptions import TimeoutException

PREFIX_STR = '<orq>Ability to '
strip_count = len(PREFIX_STR)
def do_cypher_tx(tx, verbose=False):
    cypher_str = f'''
    MATCH (np:NavigableParents)
    WHERE
        (np.navigable_parent STARTS WITH "{PREFIX_STR}")
        //AND (np.navigable_parent CONTAINS "preferred")
    RETURN np.navigable_parent AS navigable_parent; '''
    if verbose:
        clear_output(wait=True)
        print(cypher_str)
    parameter_dict = {}
    results_list = tx.run(query=cypher_str, parameters={})
    values_list = []
    for record in results_list:
        values_list.append(dict(record.items()))

    return values_list

with cu.driver.session() as session:
    rows_list = session.write_transaction(do_cypher_tx, verbose=True)
df = pd.DataFrame(rows_list)
ability2_list = df.navigable_parent.tolist()
df.shape

In [ ]:

driver = wsu.get_driver(verbose=False)
for navigable_parent in tqdm(ability2_list):
    orq_str = navigable_parent[strip_count:]
    orq_str = orq_str[0].upper() + orq_str[1:]
    orq_str = orq_str.replace('</orq>', '')
    try:
        youchat_text = wsu.get_chatgpt_rephrasing(driver, orq_str, part_of_speech='minimum requirement', verbose=False)
    except TimeoutException as e:
        clear_output(wait=True)
        driver.close()
        wsu.wait_for(1000, verbose=False)
        driver = wsu.get_driver(verbose=False)
        youchat_text = wsu.get_chatgpt_rephrasing(driver, orq_str, part_of_speech='minimum requirement', verbose=False)
    if youchat_text:
        youchat_text = '<orq>' + youchat_text + '</orq>'
        def do_cypher_tx(tx, navigable_parent, youchat_html, verbose=False):
            cypher_str = '''
                MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
                SET np.navigable_parent = $youchat_html;'''
            if verbose:
                clear_output(wait=True)
                print(cypher_str.replace('$navigable_parent', f'"{navigable_parent}"').replace('$youchat_html', f'"{youchat_html}"'))
            parameter_dict = {'navigable_parent': navigable_parent, 'youchat_html': youchat_html}
            tx.run(query=cypher_str, parameters=parameter_dict)

        with cu.driver.session() as session:
            session.write_transaction(
                do_cypher_tx, navigable_parent=navigable_parent, youchat_html=youchat_text, verbose=False
            )
driver.close()

In [7]:

driver.close()